## HPE SimpliVity Ansible module
    SimpliVity automation using Ansible
    Github repository :: https://github.com/HewlettPackard/simplivity-ansible

## Requirements

- Ansible >= 2.5</br>
- Python >= 3.3</br>
- HPE SimpliVity Python SDK</br>

## Installation
To perform a full installation, you must execute the following steps:<br>

In [ ]:
# Clone the repository from Github for the Simplivity Ansible tools
# Note that this command will fail when the folder ./simplivity-ansible already exists and there are files inside the folder.
# You will need to perform this line of code if you are working on your own environments outside of this webinar.
# git clone https://github.com/HewlettPackard/simplivity-ansible.git  #(folder for this exercise is available)

In [ ]:
# Packages required for SimpliVity Ansible module to run
# You will need to perform this line of code if you are working on you own environments outside of this webinar
# pip install -r simplivity-ansible/requirements.txt  #(packages are already installed)

In [ ]:
# Below environment variables needs to be set for adding simplivity library to the Ansible path
export ANSIBLE_MODULE_UTILS=/home/student$$I/Discover/WKSHP-SimpliVity-API/simplivity-ansible/library/module_utils
export ANSIBLE_LIBRARY=/home/student$$I/Discover/WKSHP-SimpliVity-API/simplivity-ansible/library

## SimpliVity client configuration
There are three ways to configure the simpliVity client.</br>
### 1. Using a JSON Configuration File
Save the below JSON in a file and update the details.</br>

  {
  "ip": "10.30.4.56",
  "credentials": {
    "username": "username",
    "password": "password"
    }
  }

🔒 Tip: Check the file permissions to ensure correct security, since the password is stored in clear-text.

The configuration file path must be provided using config argument.</br>
For example,

      - name: Gather facts about SimpliVity virtual machine'
        simplivity_virtual_machine_facts:
          config: "/path/to/config.json"
          name: "VM name"
          
### 2. Environment Variables

If you prefer, the configuration can also be stored in environment variables.

export SIMPLIVITYSDK_OVC_IP='10.30.4.56'</br>
export SIMPLIVITYSDK_USERNAME='username'</br>
export SIMPLIVITYSDK_PASSWORD='password'</br>

🔒 Tip: Make sure no unauthorised person has access to the environment variables, since the password is stored in clear-text.

Example,  
          
      - name: Gather facts about SimpliVity virtual machine'
        simplivity_virtual_machine_facts:
          name: "VM name"
          
### 3. Pass parameters in the playbook itself.
    
The third way to pass in your OVC credentials to your tasks is through an explicit specification on the task.</br>
This option allows the parameters ovc_ip, username, password to be passed directly inside your task.

     - name: Gather facts about SimpliVity virtual machine'
       simplivity_virtual_machine_facts:
         ovc_ip: 10.30.4.56
         username: username
         password: password
         name: "VM name"
       no_log: true
       delegate_to: localhost

**Setting no_log: true** is highly recommended in this third case, because the credentials are otherwise printed out in the log after the task completion.
    

## Create/Update a Virtual Machine using Ansible playbook
<div class="alert alert-block alert-warning">
<b>READ FIRST!</b> Note that STUDENTXX is the assigned Student ID provided to you when you join this webinar. Using the folder on the left windows, traverse the folder /SimpliVity-ansible/demo_playbooks/ and edit the vm_create_or_update.yml file
</div>

    - name: Create a virtual machine on given ESXi hostname
      vmware_guest:
        hostname: "16.31.86.185"
        username: "root"
        password: "vmware"
        validate_certs: no
        folder: Barcelona/vm
        name: student$$I_Ansible_demo_vm
        state: present
        guest_id: ubuntu64Guest
        datacenter: CCP_Virt_0001
        # This is hostname of particular ESXi server on which user wants VM to be deployed
        esxi_hostname: "16.31.87.117"
        disk:
        - size_gb: 10
          type: thick
          datastore: CCP_Virt_0001
        hardware:
          memory_mb: 256
          num_cpus: 1
        cdrom:
          type: none
        networks:
        - name: VM Network
          type: static
          device_type: flexible
      delegate_to: localhost
      register: student$$I_deploy_vm   

In [ ]:
# Create a Virtual Machine and run the playbook again if you want to update any of the VM configurations like Network, datastore,etc.
ansible-playbook -i simplivity-ansible/demo_playbooks/hosts simplivity-ansible/demo_playbooks/vm_create_or_update.yml -vvv

## SimpliVity Virtual Machine backup
<div class="alert alert-block alert-warning">
<b>READ FIRST!</b> Note that STUDENTXX is the assigned Student ID provided to you when you join this webinar. Using the folder on the left windows, traverse the folder /SimpliVity-ansible/demo_playbooks/ and edit the svt_vm_backup.yml file
</div>

     - vars:
            vm_name: student$$I_Ansible_demo_vm
    
     - name: 'Simplivity create VM backup'
        simplivity_virtual_machine:
            ovc_ip: 16.31.87.107
            username: administrator@vsphere.local
            password: vmware
            state: backup
            data:
                name: '{{ vm_name }}'
                backup_name: '{{ vm_name }}_backup1'
                cluster_name: null
                app_consistent: false
                consistency_type: null
                retention: 0
        delegate_to: localhost
    

In [ ]:
# Take simpliVity backup of a VM
ansible-playbook -i simplivity-ansible/demo_playbooks/hosts simplivity-ansible/demo_playbooks/svt_vm_backup.yml -vvv

## SimpliVity Virtual Machine clone
<div class="alert alert-block alert-warning">
<b>READ FIRST!</b> Note that STUDENTXX is the assigned Student ID provided to you when you join this webinar. Using the folder on the left windows, traverse the folder /SimpliVity-ansible/demo_playbooks/ and edit the svt_vm_clone.yml file
</div>

    - vars:
          vm_name: student$$I_Ansible_demo_vm
    - name: 'Simplivity clone'
          simplivity_virtual_machine:
            ovc_ip: 16.31.87.107
            username: root
            password: vmware
            state: clone
            data:
              name: '{{ vm_name }}'
              new_name: '{{ vm_name}}_clone'
        delegate_to: localhost

    

In [ ]:
# Take simpliVity clone of a VM
ansible-playbook -i simplivity-ansible/demo_playbooks/hosts simplivity-ansible/demo_playbooks/svt_vm_clone.yml -vvv

## SimpliVity Virtual Machine set policy
<div class="alert alert-block alert-warning">
<b>READ FIRST!</b> Note that STUDENTXX is the assigned Student ID provided to you when you join this webinar. Using the folder on the left windows, traverse the folder /SimpliVity-ansible/demo_playbooks/ and edit the svt_vm_set_policy.yml file
</div>

     - vars:
           vm_name: student$$I_Ansible_demo_vm
           policy_name: Policy_01 
     - name: 'Simplivity set VM policy'
          simplivity_virtual_machine:
            ovc_ip: 16.31.87.107
            username: root
            password: vmware
            state: set_policy
            data:
              name: '{{ vm_name }}'
              policy_name: '{{ policy_name }}'
          delegate_to: localhost
    

In [ ]:
# Set backup policy of a VM
ansible-playbook -i simplivity-ansible/demo_playbooks/hosts simplivity-ansible/demo_playbooks/svt_vm_set_policy.yml -vvv

# Continue to go back to the Introduction Notebook
Description: This link will allow you to try other notebooks.

* [Introduction](Introduction.ipynb)